# Quantum Convolutional Neural Network

In [21]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)

# Quantum Convolution Layer

In [3]:
# Convolutional Layer
def U(theta1, theta2, wires):
    qml.RY(theta1, wires = wires[0])
    qml.RY(theta2, wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

In [4]:
#Pooling Layer
def V(theta, wires):
    qml.CRZ(theta, wires = [wires[1], wires[0]])

In [15]:
# Fully connected layer
def F(wires):
    qml.CNOT(wires = [wires[0], wires[1]])

In [16]:
dev = qml.device('default.qubit', wires = 8)

In [87]:
from pennylane.templates.embeddings import AmplitudeEmbedding
@qml.qnode(dev)
def QCNN(X, params):
    AmplitudeEmbedding(X, wires = range(8), normalize = True)
    
    # 1st convolutional layer
    for i in range (0,8,2):
        U(params[0], params[1], wires = [i, i+1])
    for i in range (1,7,2):
        U(params[0], params[1], wires = [i, i+1])
    # 1st pooling layer
    for i in range(0,8,2):
        V(params[2], wires = [i, i+1])
        
    # 2nd convolution layer
    U(params[3], params[4], wires = [0,2])
    U(params[3], params[4], wires = [4,6])
    U(params[3], params[4], wires = [2,4])
    
    # 2nd pooling layer
    V(params[5], wires = [0, 2])
    V(params[5], wires = [4, 6])
    
    #Fully Connected Layer
    F(wires = [0,4])
    return qml.expval(qml.PauliZ(4))

In [18]:
params_init = np.random.randn(6)
print(params_init)

[-8.59731786e-04  1.07296077e+00  1.56414876e-01  6.08737328e-02
 -1.19747830e+00  6.06970507e-01]


In [20]:
circuit = QCNN(params_init)
print(circuit)

0.13420148978046242


# Training QCNN

In [27]:
from tensorflow.keras import datasets, layers, models
import tensorflow as tf

In [126]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(params, X, Y):
    predictions = [QCNN(x, params) for x in X]
    return square_loss(Y, predictions)

def accuracy(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 1e-3:
            acc = acc + 1
    return acc / len(labels)

In [115]:
def data_load_and_process(classes = [0,1]):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    
    x_train_01 = x_train[x_train_filter_01]
    y_train_01 = y_train[x_train_filter_01]
    x_test_01 = x_test[x_test_filter_01]
    y_test_01 = y_test[x_test_filter_01]
    
    
    x_train= tf.cast(x_train_01, tf.float32)
    x_test=tf.cast(x_test_01, tf.float32)
    
    x_train = tf.image.resize(x_train[:], (256, 1)).numpy()
    x_test = tf.image.resize(x_test[:], (256, 1)).numpy()
    
    x_train, x_test = tf.squeeze(x_train), tf.squeeze(x_test)
                                                                            
    
    
    return x_train, x_test, y_train_01, y_test_01

In [116]:
x_train, x_test, y_train, y_test = data_load_and_process()

In [124]:
def optimization(X_train, Y_train):
    params = np.random.randn(6)
    steps = 100
    learning_rate = 0.1
    batch_size =25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch), params)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return params
    

In [125]:
params_trained = optimization(x_train, y_train)

iteration:  0  cost:  0.5172121658070388
iteration:  10  cost:  0.3329628060288966
iteration:  20  cost:  0.3648150483798443
iteration:  30  cost:  0.32441461563751156
iteration:  40  cost:  0.21544728273759184
iteration:  50  cost:  0.2826444392577369
iteration:  60  cost:  0.18101653190288572
iteration:  70  cost:  0.11723825593607018
iteration:  80  cost:  0.20214640867191336
iteration:  90  cost:  0.24528610079952165
